```markdown
### Purpose of this Notebook

This notebook demonstrates the use of various tools and libraries to interact with OpenAI's language models and LangChain for natural language processing tasks. The key objectives include:

1. **Setting up OpenAI API**: Installing necessary packages, retrieving API keys, and initializing the OpenAI client.
2. **Creating and using chat functions**: Defining functions to interact with the language model for generating responses based on user prompts.
3. **Translating text styles**: Using the language model to translate text into different styles.
4. **LangChain Integration**: Installing and using LangChain to create structured prompts and parse responses from the language model.
5. **Output Parsing**: Defining response schemas and using LangChain formatters to automatically generate and parse structured outputs from the language model.
6. **Extracting insights from customer reviews**: Formatting prompts to extract specific information from customer reviews and parsing the model's responses into structured data.

This notebook provides examples of how to leverage OpenAI's language models and LangChain for various NLP tasks, including text translation, structured output generation, and information extraction.
```

In [ ]:
%pip install python-dotenv
%pip install openai

In [ ]:
import subprocess

OPEN_AI_KEY = subprocess.run('bash -l -c "getpassw $OPEN_AI_API_KEY"', shell=True, capture_output=True, text=True)
OPEN_AI_KEY = OPEN_AI_KEY.stdout.strip()
print("")

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key=OPEN_AI_KEY
)

In [4]:
LLM_MODEL = "gpt-4o-mini-2024-07-18"
def chat(prompt, model=LLM_MODEL):
    messages = [{"role": "user", "content": prompt}]
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return chat_completion.choices[0].message.content

In [5]:
chat("what is 1 +    1")

'1 + 1 equals 2.'

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone

text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
chat(prompt)

'I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you!'

## Chat API : LangChain

In [ ]:
%pip install -U langchain-openai
%pip install langchain_community


In [ ]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(temperature=0.0, model=LLM_MODEL, openai_api_key=OPEN_AI_KEY)
print("")

### Prompt template

In [10]:
prompt_template = """Translate the text \
that is delimited by triple backticks \
into a style that is {style} \
text: ```{text}````
"""
print(prompt_template)

Translate the text that is delimited by triple backticks into a style that is {style} text: ```{text}````



In [11]:
from langchain.prompts import ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template)
print(chat_prompt_template)

input_variables=['style', 'text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style} text: ```{text}````\n'))]


In [12]:
chat_prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style} text: ```{text}````\n')

In [13]:
style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = chat_prompt_template.format_messages(style=style, text=customer_email)

In [15]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
 text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
````



In [ ]:
response = chat(customer_messages)

In [17]:
print(response)

content='I’m quite frustrated that the lid of my blender came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn’t cover the cost of cleaning up my kitchen. I would really appreciate your assistance with this issue. Thank you!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 87, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-25d717fd-67b7-4fa1-8f30-35082e88fbaa-0' usage_metadata={'input_tokens': 87, 'output_tokens': 54, 'total_tokens': 141}


In [18]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = chat_prompt_template.format_messages(style=service_style_pirate, text=service_reply)

In [19]:
print(service_messages)

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n````\n")]


In [20]:
service_response = chat(service_messages)

In [21]:
print(service_response.content)

Ahoy there, esteemed customer! I must inform ye that the warranty be not coverin' the expenses for cleanin' yer galley, as it be yer own misfortune that ye forgot to secure the lid upon yer blender before settin' it to whirl. Aye, 'tis a tough break! Fair winds to ye, and until we meet again!


### Using LangChain Output Parser to parse the output of the llm

In [22]:
from langchain.prompts import ChatPromptTemplate

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the customer_review delimited by triple backticks, \
extract the following information: \

gift: Was this item a gift for someone else? \
Answer True if yes and False if otherwise. \

delivery_time: How many days did it take for the item to arrive? \
Answer with a number, and if the item has not arrived or if it is not known then answer with -1. \

price: Extract the sentences about value or price \
and output the amounts in a comma sep python list. \

Format the output as a JSON object with following keys: \
gift, delivery_time, price. \

customer_review: {customer_review}
"""

customer_review_template = ChatPromptTemplate.from_template(review_template)
get_insights_from_customer_review = customer_review_template.format_messages(customer_review=customer_review)
print(get_insights_from_customer_review[0].content)

For the customer_review delimited by triple backticks, extract the following information: 
gift: Was this item a gift for someone else? Answer True if yes and False if otherwise. 
delivery_time: How many days did it take for the item to arrive? Answer with a number, and if the item has not arrived or if it is not known then answer with -1. 
price: Extract the sentences about value or price and output the amounts in a comma sep python list. 
Format the output as a JSON object with following keys: gift, delivery_time, price. 
customer_review: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for t

In [23]:
response = chat(get_insights_from_customer_review)

In [24]:
print(response.content)

```json
{
  "gift": true,
  "delivery_time": 2,
  "price": ["slightly more expensive"]
}
```


In [25]:
print(type(response.content))

<class 'str'>


### LangChain Formatters allows to format the llm output directly
#### We have to define the response schema and use LangChain formatters to automatically generate the response prompt for us.

In [111]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name= "gift", type="boolean", 
                             description=(
                                "Was this item a gift for someone else? "
                                "Answer True if yes and False if otherwise. " 
                             )
)
delivery_time_schema = ResponseSchema(name= "delivery_time", type="integer", 
                             description=(
                              "How many days did it took for the item to be delivered? "
                              "Answer -1 if you don't know or if the item has not arrived yet. "   
                             ) 
)

price_schema = ResponseSchema(name="price", type="json", 
                             description=(
                                 "First analyze the customer's opinion "
                                 "by extracting the sentences related to price or value. "
                                 "Then, assess whether the customer finds the product expensive, moderate or affordable. "
                                 "Ensure that your final conclusion should reflect the customer's true attitude specifically towards price, taking into account any justification or perceived worth. "
                                 "Answer with a final conclusion (expensive, moderate or affordable) and reason for conclusion in a JSON object with keys conclusion, reason_for_conclusion. "
                             )
)
output_format_instructions = StructuredOutputParser.from_response_schemas([gift_schema, delivery_time_schema, price_schema])


In [112]:
print(output_format_instructions.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": boolean  // Was this item a gift for someone else? Answer True if yes and False if otherwise. 
	"delivery_time": integer  // How many days did it took for the item to be delivered? Answer -1 if you don't know or if the item has not arrived yet. 
	"price": json  // First analyze the customer's opinion by extracting the sentences related to price or value. Then, assess whether the customer finds the product expensive, moderate or affordable. Ensure that your final conclusion should reflect the customer's true attitude specifically towards price, taking into account any justification or perceived worth. Answer with a final conclusion (expensive, moderate or affordable) and reason for conclusion in a JSON object with keys conclusion, reason_for_conclusion. 
}
```


In [114]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the customer_review delimited by triple backticks, \
extract the following information: \

gift: {gift_instruction}

delivery_time: {delivery_time_instruction}

price: {price_instruction}

Format the output as a JSON object with following keys: \
gift, delivery_time, price. \

customer_review: {customer_review}

{output_format_instructions}
"""

customer_review_template = ChatPromptTemplate.from_template(review_template)

get_insights_from_customer_review = \
    customer_review_template.format_messages(
        gift_instruction=gift_schema.description,
        delivery_time_instruction=delivery_time_schema.description,
        price_instruction=price_schema.description,
        customer_review=customer_review,
        output_format_instructions=output_format_instructions.get_format_instructions()
    )

In [116]:
print(get_insights_from_customer_review[0].content)

For the customer_review delimited by triple backticks, extract the following information: 
gift: Was this item a gift for someone else? Answer True if yes and False if otherwise. 

delivery_time: How many days did it took for the item to be delivered? Answer -1 if you don't know or if the item has not arrived yet. 

price: First analyze the customer's opinion by extracting the sentences related to price or value. Then, assess whether the customer finds the product expensive, moderate or affordable. Ensure that your final conclusion should reflect the customer's true attitude specifically towards price, taking into account any justification or perceived worth. Answer with a final conclusion (expensive, moderate or affordable) and reason for conclusion in a JSON object with keys conclusion, reason_for_conclusion. 

Format the output as a JSON object with following keys: gift, delivery_time, price. 
customer_review: This leaf blower is pretty amazing.  It has four settings:candle blower, 

In [117]:
response = chat(get_insights_from_customer_review)

In [118]:
print(response.content)

```json
{
	"gift": true,
	"delivery_time": 2,
	"price": {
		"conclusion": "expensive",
		"reason_for_conclusion": "The customer mentions that it is slightly more expensive than other leaf blowers, indicating a perception of higher cost compared to alternatives."
	}
}
```


In [119]:
customer_review_obj = output_format_instructions.parse(response.content)
print(type(customer_review_obj))

<class 'dict'>


In [122]:
print(customer_review_obj["gift"])
print(customer_review_obj["delivery_time"])
print(customer_review_obj["price"])


True
2
{'conclusion': 'expensive', 'reason_for_conclusion': 'The customer mentions that it is slightly more expensive than other leaf blowers, indicating a perception of higher cost compared to alternatives.'}
